In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../src')

from dataset import DatasetConfig, DatasetBuilder
import os
from functools import partial
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import defaultdict
from typing import Callable, Dict, List, Tuple
%matplotlib inline

In [2]:
ds_config =  DatasetConfig(model_dir='../dataset/pinecone/sparse/0/',
                           images_dir='../dataset/pinecone/images/',
                           batch_per_image=4,
                           is_training=True,
                           batch_size=256)

In [3]:
dataset = DatasetBuilder(ds_config)
ds = dataset.build()

In [5]:
lines = []
colors = []

def make_line(origin, direction, length=10):
    return origin, origin + direction * length

for elements in ds.take(100).as_numpy_iterator():
    origins = elements["origins"]
    directions = elements["directions"]
    pixels = elements["pixels"]
    N = origins.shape[0]
    for i in range(N):
        p0, p1 = make_line(origins[i, ...], directions[i, ...])
        lines.append(p0)
        lines.append(p1)
        colors.append(pixels[i, ...])

In [6]:
data = np.asarray(lines)
np.save('lines.npy', data)

colors = np.asarray(colors)
np.save('colors.npy', colors)

In [5]:
origins

array([[-2.804063  ,  2.7937622 ,  1.1826246 ],
       [-2.60579   ,  2.244224  , -0.3216559 ],
       [-3.07649   ,  1.6968286 ,  1.1726636 ],
       [-2.80071   ,  2.3194833 ,  0.17904912],
       [-2.0700405 ,  3.6704292 ,  1.0835867 ],
       [-2.0700405 ,  3.6704292 ,  1.0835867 ],
       [ 3.7744408 ,  1.8780159 , -1.4483403 ],
       [-2.0700405 ,  3.6704292 ,  1.0835867 ],
       [-2.0700405 ,  3.6704292 ,  1.0835867 ],
       [-2.5361595 ,  2.1788397 , -0.9649486 ],
       [ 3.7744408 ,  1.8780159 , -1.4483403 ],
       [-2.5361595 ,  2.1788397 , -0.9649486 ],
       [-2.0700405 ,  3.6704292 ,  1.0835867 ],
       [-3.07649   ,  1.6968286 ,  1.1726636 ],
       [-2.0700405 ,  3.6704292 ,  1.0835867 ],
       [-2.80071   ,  2.3194833 ,  0.17904912],
       [-2.80071   ,  2.3194833 ,  0.17904912],
       [-2.0700405 ,  3.6704292 ,  1.0835867 ],
       [-3.07649   ,  1.6968286 ,  1.1726636 ],
       [-0.23490237,  2.0134726 , -2.1048646 ],
       [-2.0700405 ,  3.6704292 ,  1.083